In [2]:
import json
import os
import pandas as pd
from tqdm import tqdm
import pickle
import gzip
import re
from getpass import getpass
from openai import OpenAI

NOME_ARQUIVO_RESULTADOS_PESQUISAS_TODOS_CHUNKS = 'outputs/3 - resultados_pesquisas/resultados_pesquisas_todos_chunks.pickle.gz'
NOME_ARQUIVO_RESULTADOS_PESQUISAS_APENAS_ART = 'outputs/3 - resultados_pesquisas/resultados_pesquisas_apenas_art.pickle.gz'

# Para abrir os arquivos depois:
def load_pickle_gzip(path):
    import pickle
    import gzip
    with gzip.open(path, 'rb') as f:
        return pickle.load(f)

resultados_pesquisas_todos_chunks = load_pickle_gzip(NOME_ARQUIVO_RESULTADOS_PESQUISAS_TODOS_CHUNKS)
resultados_pesquisas_apenas_art = load_pickle_gzip(NOME_ARQUIVO_RESULTADOS_PESQUISAS_APENAS_ART)

In [5]:
resultados_pesquisas_todos_chunks['1'].keys()

dict_keys(['bm25', 'text-embedding-3-large'])

In [ ]:
total_porcentagem = len(EXPERIMENTOS)*len(questoes)
with tqdm(total=total_porcentagem) as pbar:
    for experimento in EXPERIMENTOS:
        for questao in questoes:
            id_questao = questao['ID_QUESTAO']
            
            # 1) SEM NENHUM CONTEXTO
            modelo_rag = ''
            tipo_pesquisa_chunks = ''
            urns_chunks = []
            executa_experimento_e_atualiza(df_resultados_experimentos, experimento, questao, modelo_rag, tipo_pesquisa_chunks, urns_chunks)
    
            # 2) COM TODO O CONTEXTO DO DATASET CONSIDERANDO TODOS OS CHUNKS
            modelo_rag = 'Gold'
            tipo_pesquisa_chunks = 'todos_chunks'
            urns_chunks = questao['URN_FUNDAMENTACAO']
            executa_experimento_e_atualiza(df_resultados_experimentos, experimento, questao, modelo_rag, tipo_pesquisa_chunks, urns_chunks)
    
            # 3) COM TODO O CONTEXTO DO DATASET CONSIDERANDO TODOS OS CHUNKS APENAS DE ARTIGOS COMPLETOS OU JURISPRUDENCIA
            modelo_rag = 'Gold'
            tipo_pesquisa_chunks = 'apenas_art'
            urns_chunks = questao['URN_FUNDAMENTACAO_APENAS_ART']
            executa_experimento_e_atualiza(df_resultados_experimentos, experimento, questao, modelo_rag, tipo_pesquisa_chunks, urns_chunks)

            # 4) CONSIDERANDO DE 1 A 3 CHUNKS PARA CADA MODELO DE RAG E CONSIDERANDO TODOS OS CHUNKS
            tipo_pesquisa_chunks = 'todos_chunks'
            modelos_rag_disponiveis_para_questao = resultados_pesquisas_todos_chunks[id_questao].keys()
            for modelo_rag in modelos_rag_disponiveis_para_questao:
                for n_chunks in range(1, MAX_CHUNKS_PARA_TESTAR+1):
                    urns_chunks = list(resultados_pesquisas_todos_chunks[id_questao][modelo_rag]['urn'][0:n_chunks]))
                    executa_experimento_e_atualiza(df_resultados_experimentos, experimento, questao, modelo_rag, tipo_pesquisa_chunks, urns_chunks)
                    
            # 5) CONSIDERANDO DE 1 A 3 CHUNKS PARA CADA MODELO DE RAG E CONSIDERANDO CHUNKS DE APENAS ARTIGOS
            tipo_pesquisa_chunks = 'apenas_art'
            modelos_rag_disponiveis_para_questao = resultados_pesquisas_apenas_art[id_questao].keys()
            for modelo_rag in modelos_rag_disponiveis_para_questao:
                for n_chunks in range(1, MAX_CHUNKS_PARA_TESTAR+1):
                    urns_chunks = list(resultados_pesquisas_apenas_art[id_questao][modelo_rag]['urn'][0:n_chunks]))
                    executa_experimento_e_atualiza(df_resultados_experimentos, experimento, questao, modelo_rag, tipo_pesquisa_chunks, urns_chunks)
            
            pbar.update(1)

In [24]:
import json
from collections import defaultdict

arquivo = "temp.jsonl"  # caminho do seu arquivo

respostas_por_questao = defaultdict(set)

# Ler o jsonl
with open(arquivo, "r", encoding="utf-8") as f:
    for linha in f:
        dado = json.loads(linha)
        id_questao = dado["ID_QUESTAO"]
        resposta = dado["RESPOSTA_LLM"]
        respostas_por_questao[id_questao].add(resposta)

# Verificar quais têm respostas diferentes
ids_com_respostas_diferentes = [
    id_questao
    for id_questao, respostas in respostas_por_questao.items()
    if len(respostas) > 1
]

print("ID_QUESTAO com respostas diferentes:")
for id_questao in ids_com_respostas_diferentes:
    print(id_questao)


ID_QUESTAO com respostas diferentes:
8
28
41
56
61
67
134
220
254
283
